<a href="https://colab.research.google.com/github/SebastianLarssonDTU/02456-Reinforcement-Learning-Project/blob/before_framestacking/Generate_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INIT : Procgen, Drive, Git

In [ ]:
!pip install procgen

     |████████████████████████████████| 39.9MB 112kB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
     |████████████████████████████████| 665kB 57.0MB/s 
     |████████████████████████████████| 22.2MB 62.5MB/s 
     |████████████████████████████████| 204kB 49.2MB/s 
     |████████████████████████████████| 3.3MB 39.9MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: imageio 2.4.1
    Uninstalling imageio-2.4.1:
      Successfully uninstalled imageio-2.4.1


In [ ]:
#Clone git
!git clone -b before_framestacking https://github.com/SebastianLarssonDTU/02456-Reinforcement-Learning-Project.git "my_project"

Cloning into 'my_project'...
remote: Enumerating objects: 143, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (143/143), done.
remote: Total 603 (delta 86), reused 2 (delta 0), pack-reused 460
Receiving objects: 100% (603/603), 9.09 MiB | 15.94 MiB/s, done.
Resolving deltas: 100% (369/369), done.


In [ ]:
#update git
%cd /content/my_project
! git pull

/content/my_project
Already up to date.


In [ ]:
import datatools as tools
from datatools import DATA_PATH, MODEL_PATH
#Mount drive
tools.mount_drive()

Mounted at /content/drive


# TESTING

In [ ]:
#Import all custom files
import baseline
import datatools as tools
import hyperparameters as h
import model
import my_util
import policy
import ppo
import utils

#other imports
import torch

#import specific methods
from baseline import set_hyperparameters
from ppo import PPO
from experiments import run_experiment, print_list_of_experiments


In [ ]:
# run_experiment(2, levels=50, save_interval=5e5)

In [ ]:
# #for Experiment 7
# file_name = "Experiment7_50levels_Run_04Dec_18h12m45s_loaded_05Dec_10h49m07s_13516800steps"
# set_hyperparameters(baseline="Impala")
# h.value_clipping = True
# h.num_levels = 50
# h.batch_size=512
# model = PPO(print_output=True, eval=True, save_interval=5e5)
# model.load_policy(file_name)
# model.train()

In [ ]:
# #For Experiment 2
# file_name = "Experiment2_50levels_Run_04Dec_18h10m32s_loaded_05Dec_09h43m36s_13516800steps"
# set_hyperparameters(baseline="PPO")
# h.value_clipping = True
# h.num_levels = 50
# model = PPO(print_output=True, eval=True, save_interval=5e5)
# model.load_policy(file_name)
# model.train()

In [ ]:
# #for Experiment 8
# file_name = "Experiment8_50levels_Run_05Dec_08h34m02s_4005888steps"
# set_hyperparameters(baseline="Impala")
# h.value_clipping = True
# h.num_levels = 50
# h.batch_size=512
# h.death_penalty = True
# h.penalty = 5
# model = PPO(print_output=True, eval=True, save_interval=5e5)
# model.load_policy(file_name)
# model.train()

# Generate video

In [ ]:
def old_policy_evaluation(model, video_name=None, print_output=False):
  policy = model.policy

  if video_name is None:
    video_name = model.file_name
  # Make evaluation environment
  eval_env = make_env(num_envs, start_level=num_levels, num_levels=num_levels)
  obs = eval_env.reset()

  frames = []
  total_reward = []

  # Evaluate policy
  policy.eval()
  for _ in range(512):  #<--- Hardcoded batch size?

    # Use policy
    action, log_prob, value = policy.act(obs)

    # Take step in environment
    obs, reward, done, info = eval_env.step(action)
    total_reward.append(torch.Tensor(reward))

    # Render environment and store
    frame = (torch.Tensor(eval_env.render(mode='rgb_array'))*255.).byte()
    frames.append(frame)

  # Calculate average return
  total_reward = torch.stack(total_reward).sum(0).mean(0)
  
  if print_output:
    print('Average return:', total_reward)

  # Save frames as video
  frames = torch.stack(frames).cpu().numpy()
  imageio.mimsave(video_name+'.mp4', frames, fps=25)
  
  return total_reward

## Load model

In [ ]:
#for Experiment 8
file_name = "Experiment8_50levels_Run_05Dec_08h34m02s_loaded_07Dec_10h53m55s_7512064steps"
set_hyperparameters(baseline="Impala")
h.value_clipping = True
h.num_levels = 50
h.batch_size=512
h.death_penalty = True
h.penalty = 5
model = PPO(print_output=True, eval=True, save_interval=5e5)
model.load_policy(file_name)

RuntimeError: ignored

In [ ]:
old_policy_evaluation(model)